In [16]:
import heapq
from pathlib import Path

In [17]:
ENERGY = dict(A=1, B=10, C=100, D=1000)
TARGETS = {"A": 3, "B": 5, "C": 7, "D": 9}

data = Path("day23.txt").read_text()

In [21]:

def parse(string):
    return {(i, j): c
            for i, l in enumerate(string.splitlines()) 
            for j, c in enumerate(l)
            if c in "ABCD"}

def render(state):
    height = max(k[0] for k in state.keys())
    return "\n".join("".join(state.get((i, j), 
                                       "." if i == 1 and j in range(1, 12) 
                                           or i in range(1, height + 1) and j in {3, 5, 7, 9} 
                                       else "#") 
                             for j in range(13)) 
                     for i in range(height + 2))

def update(state, old, new, value):
    state = state.copy()
    state.pop(old)
    state[new] = value
    return state

def expand(queue, done, state, cost):
    state = render(state)
    if cost < done.get(state, 1e6):
        done[state] = cost
        heapq.heappush(queue, (cost, state))

def solve(state, depth):
    done = {}
    queue = [(0, state)]
    while queue:
        # basic dijkstra, pull next cheapest state off the heap
        cost, state = heapq.heappop(queue)
        state = parse(state)

        # if we have rearranged all the amphipods, return the cheapest cost
        if all(state.get((i, j)) == a for a, j in TARGETS.items() for i in range(2, depth + 2)):
            return cost

        # attempt to expand the search for all amphipods
        for (i, j), a in state.items():
            pcost = cost
            p = i
            
            # first, see if this amphipod can leave its current room
            # the amphipod will stay put if there are any others blocking
            # it or if it has reached its final destination
            if i > 1 and all((k, j) not in state for k in range(2, i)):
                if j != TARGETS[a] or not all(state.get((k, j), a) == a for k in range(2, depth + 2)):
                    p = 1
                    pcost += abs(p - i) * ENERGY[a]

            # if we are in the hallway, move left/right until stopped
            if p == 1:
                for qs in [range(j - 1, 0, -1), range(j + 1, 12)]:
                    for q in qs:
                        # if we have reached another amphipod, stop
                        if (p, q) in state:
                            break
                            
                        # if we are just entering the hallway (rule 3), 
                        # expand the search in all non-doorway steps
                        if i > 1 and q not in {3, 5, 7, 9}:
                            expand(queue, 
                                   done, 
                                   update(state, (i, j), (p, q), a),
                                   pcost + abs(q - j) * ENERGY[a])
                        
                        # if we have reached our destination room
                        # and it only contains its target amphipods,
                        # move into it
                        if q == TARGETS[a] and all(state.get((k, q), a) == a for k in range(2, depth + 2)):
                            t = max(k for k in range(2, depth + 2) if (k, q) not in state)
                            expand(queue,
                                   done,
                                   update(state, (i, j), (t, q), a),
                                   pcost + (abs(t - 1) + abs(q - j)) * ENERGY[a])


## Part 1

In [22]:
solve(data, depth=2)

11516

## Part 2

In [23]:
data2 = render(parse("\n".join(data.splitlines()[:3] + 
                               ["  #D#C#B#A#", 
                                "  #D#B#A#C#"] +
                               data.splitlines()[3:])))
solve(data2, depth=4)

40272